In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve,least_squares,root

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [2]:
p = RollRig.base_params(3)

In [4]:
sm = mf.SymbolicModel.from_file('RollRigModel.py')
sm.ExtForces = ef.CompositeForce([sm.ExtForces,ef.CustomForce(None)])
sm = sm.subs({p.alpha_r.name:0,p.alpha_1.name:sym.atan(sym.sin(p.Lambda)*sym.sin(p.q[1])),
             p.alpha_2.name:sym.atan(sym.sin(p.Lambda)*sym.sin(p.q[2]))})

In [5]:
p.c.value = 0.067
p.s.value = 1
p.sigma.value = 0.28
p.m_w.value = 0.22
p.m_f.value = 0.038
p.m_l.value = 0#0.0275
p.alpha_r.value = 0
p.c_d_max.value = 1
p.a_0.value = 2*np.pi
p.a_1.value = 2*np.pi
#p.tau.value = 0
p.alpha_r.value = 0
p.w_g.value = 0
p.beta.value = 0.0036
p.V.value = 22.5

#p.s.value = 0.72
#p.sigma.value = 0
#p.m_f.value = 0
#p.m_l.value = 0


p.Lambda.value = np.deg2rad(10)

In [6]:
nm = mf.NumericModel.from_SymbolicModel(p,sm)
ext_f = nm.ExtForces.force_funcs[0]

torque_period = 0.1
torque_max = p.beta.value*p.V.value**2

def torque(tup,x,t):
    if t<2:
        return 0
    elif t<torque_period/2+2:
        return torque_max*0.5*(1- np.cos(2*np.pi*(t-2)/torque_period))
    else:
        return torque_max
    
def custom_force(tup,x,t):
    return np.array([[torque(tup,x,t)],[0],[0]])

nm.ExtForces.force_funcs = [ext_f , custom_force]



In [7]:
def GenRunData(model,p,ic,end_time,xNames=None,additional_cols = {},sample_freq=100):
    
    if xNames==None:
        xNames = [f'x{i}' for i in range(p.qs*2)]
    
    # create objective function and solve
    def objectiveDeriv(t,y):
        return nm.deriv(t,y,p.GetNumericTuple(y,t))
    yData = solve_ivp(objectiveDeriv,(0,end_time),ic,max_step=1/sample_freq)
    
    t = np.linspace(0,end_time,(end_time*100)+1)
   
    #interpolate to reduce t and y points
    int_func = interp1d(yData.t,yData.y)  
    yi = int_func(t)
    
    #generate list
    tau = [torque((0,),[0]*p.qs*2,i) for i in t]
    q_data = [{names[i]:v for i,v in enumerate(row)}for row in yi.T]    
    params = {k.name:v for k,v in p.GetSubs(0,None).items()}


    data = [{'t':_t,'torque':tau[i],'Locked':True,**q_data[i],**params,**additional_cols} for i,_t in enumerate(t)]
    return data
    

In [13]:
names = ['Roll','Roll Rate','Left FWT Angle','Left FWT Velocity','Right FWT Angle','Right FWT Velocity']

res = []

for lam in np.deg2rad([10,20,30]):
    p.Lambda.value = lam
    for camber in np.deg2rad([0,5,10]):
        p.alpha_c.value = camber

        for v in np.linspace(15,30,7):
            p.V.value = v
            ext_f = nm.ExtForces.force_funcs[0]

            torque_period = 0.1
            torque_max = p.beta.value*p.V.value**2

            def torque(tup,x,t):
                if t<2:
                    return 0
                elif t<torque_period/2+2:
                    return torque_max*0.5*(1- np.cos(2*np.pi*(t-2)/torque_period))
                else:
                    return torque_max

            def custom_force(tup,x,t):
                return np.array([[torque(tup,x,t)],[0],[0]])

            nm.ExtForces.force_funcs = [ext_f , custom_force]

            res+=GenRunData(nm,p,[0]*p.qs*2,10,names,{'Mode':'Free'},100)

In [15]:
res[0]

{'t': 0.0,
 'torque': 0,
 'Locked': True,
 'Roll': 0.0,
 'Roll Rate': 0.0,
 'Left FWT Angle': 0.0,
 'Left FWT Velocity': 0.0,
 'Right FWT Angle': 0.0,
 'Right FWT Velocity': 0.0,
 'c': 0.067,
 's': 1,
 'sigma': 0.28,
 'm_w': 0.22,
 'm_f': 0.038,
 'm_l': 0,
 'rho': 1.225,
 'V': 15.0,
 'g': 9.81,
 'alpha_r': 0,
 'alpha_c': 0.0,
 'alpha_0': 0,
 'alphadot_0': 0,
 'alpha_1': 0,
 'alphadot_1': 0,
 'alpha_2': 0,
 'alphadot_2': 0,
 'C_Dmax': 1,
 'a_0': 6.283185307179586,
 'a_1': 6.283185307179586,
 'beta': 0.0036,
 'Lambda': 0.17453292519943295,
 'w_g': 0,
 'qtilde0': 0,
 'qtilde1': 0,
 'qtilde2': 0,
 'qtilde3': 0,
 'qtilde4': 0,
 'qtilde5': 0,
 'Mode': 'Free'}

In [14]:
df = pd.DataFrame(res)
df.to_pickle('FreeData.pkl')

In [10]:
df['Roll Rate [Deg]']=np.rad2deg(df['Roll Rate'])
df[df['t']>4].groupby(['V'])['Roll Rate [Deg]'].mean()

V
15.0    314.190322
17.5    367.973462
20.0    421.435284
22.5    474.615103
25.0    527.651175
27.5    580.629241
30.0    633.556032
Name: Roll Rate [Deg], dtype: float64